In [45]:
import requests
from selenium import webdriver
import bs4
import re
from tqdm import tqdm
import gc
import pandas as pd

In [46]:
# web driver for selenium
driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')

# get the web page and store it as a soup element
driver.get('https://www.hdfcbank.com/personal/pay/cards/credit-cards')
soup = bs4.BeautifulSoup(driver.page_source, 'lxml')
driver.close()

C:\Users\manng\AppData\Local\Temp\ipykernel_30240\1210421472.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')


In [47]:
# web driver for selenium
driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')

# get the web page and store it as a soup element
driver.get('https://www.hdfcbank.com/personal/pay/cards/credit-cards')
soup = bs4.BeautifulSoup(driver.page_source, 'lxml')
driver.close()

C:\Users\manng\AppData\Local\Temp\ipykernel_30240\1210421472.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')


In [54]:
# Here we are extracting the card names and the links to the feature pages
credit_card_names = []
links = []
for i in soup.findAll("div",{"class":"cardparent"}):
    
    # From the parent class we need to extract the title/card name 
    s = i.findAll('h2',{'class':'cardTitle'})[0]
    credit_card_names.append(s.get_text('^^',strip=True).split('^^')[0])
    
    # Now we will find the links to the know more section and store them
    for link in i.findAll("a",{'href': re.compile(r'/personal'),'target':'_self'}):
        a = link.get('href')
        if a!=None:
            links.append(a)
            break

# Printing out some of the card names and their linnk
for name, link in zip(credit_card_names[:60], links[:60]):
    print(name,":",link)

MoneyBack+ Credit Card : /personal/pay/cards/credit-cards/moneyback-plus
Millennia Credit Card : /personal/pay/cards/millennia-cards/millennia-cc-new
IndianOil HDFC Bank Credit Card : /personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card
Regalia Gold Credit Card : /personal/pay/cards/credit-cards/regalia-gold-credit-card
IRCTC HDFC Bank Credit Card : /personal/pay/cards/credit-cards/irctc-credit-card
Tata Neu Plus HDFC Bank Credit Card : /personal/pay/cards/credit-cards/tata-neu-plus-hdfc-bank-credit-card
Tata Neu Infinity HDFC Bank Credit Card : /personal/pay/cards/credit-cards/tata-neu-infinity-hdfc-bank-credit-card
HDFC Bank UPI RuPay Credit Card : /personal/pay/cards/credit-cards/hdfc-bank-upi-rupay-credit-card
INFINIA Metal Edition : /personal/pay/cards/credit-cards/infinia-credit-card
Diners Club Black Credit Card : /personal/pay/cards/credit-cards/diners-club-black
Regalia Gold Credit Card : /personal/pay/cards/credit-cards/regalia-gold-credit-card
Millennia Credit Ca

In [55]:
c_links = [f"https://www.hdfcbank.com{link}" for link in links]
print(*c_links[0:60],sep='\n')

https://www.hdfcbank.com/personal/pay/cards/credit-cards/moneyback-plus
https://www.hdfcbank.com/personal/pay/cards/millennia-cards/millennia-cc-new
https://www.hdfcbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/irctc-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/tata-neu-plus-hdfc-bank-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/tata-neu-infinity-hdfc-bank-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/hdfc-bank-upi-rupay-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-club-black
https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card
https://www.hdfcbank.com/personal/pay/cards/millennia-cards/millennia-cc-new
https://www.hdf

In [56]:

def get_page(link):
    driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')
    driver.get(link)
    soup_res = bs4.BeautifulSoup(driver.page_source, 'lxml')
    driver.close()
    
    return soup_res

In [57]:

all_features = []
all_points = []
lounge_access = []
milestone_benefit = []
rewards = []
opt_s = []

for link in c_links:
    
    key_features = get_page(link)
    
    features = []      
    fpoints = []       

    for i in key_features.findAll("div",{"class":"content-body"}):
        
        s = i.findAll('h4',{'class':'row-name'})  
        
        points = i.findAll(['li','p'])
        
        tmp_points = []     
        for point in points:
            tmp_points.append(point.get_text(strip=True))
        fpoints.append(tmp_points)
        del tmp_points; gc.collect()

        try:
            features.append(s[0].get_text('^^',strip=True))

        except Exception as e:
            pass
    
    
    features = [re.sub(r'[^a-zA-Z0-9 ]+',"",f).lower() for f in features]
    
    data = {f:x for f,x in zip(features, fpoints)}
    
    filtered_featues = list(filter(lambda x: re.search('feature\w*|mile\w*|reward\w*|lounge\w*|benefit\w+',x), features))
    
    lounge_key = list(filter(lambda x:True if re.search(r'lounge',x) else False,filtered_featues))
    if len(lounge_key)!=0:
        lounge_access.append(data[lounge_key[0]])
    else:
        lounge_access.append('None')
    
    mile_key = list(filter(lambda x:True if re.search(r'mile\w*',x) else False, filtered_featues))
    if len(mile_key)!=0:
        milestone_benefit.append(data[mile_key[0]])
    else:
        milestone_benefit.append('None')
    
    rew_key = list(filter(lambda x:True if re.search(r'reward\w*|key\w*|feature\w*',x) else False,filtered_featues))
    if len(rew_key)==1:
        rewards.append(data[rew_key[0]])
    elif len(rew_key)>1:
        tmp = []
        for rk in rew_key:
            tmp.extend(data[rk])
        rewards.append(tmp)
    else:
        rewards.append('None')
    
    fee_ = get_page(link+'/fees-and-charges')
    s = []
    for i in fee_.findAll('div',{'class':'inner-content'}):
        s_ = i.findAll(['li','p'])
        s.extend([y.get_text('^^',strip=True) for y in s_])
    
    ren_off = list(filter(lambda x:True if re.search(r'renew\w*',x) else False,filtered_featues))
    if len(ren_off)!=0:
        s.extend(ren_off[0])
    opt_s.append(s)

C:\Users\manng\AppData\Local\Temp\ipykernel_30240\492962142.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')


In [58]:

detail = {"credit_card_name":credit_card_names,
    "lounge_access":lounge_access,
        "milestone":milestone_benefit,
        "rewards":rewards,
       "fees":opt_s}
df = pd.DataFrame.from_dict(detail)
df = df.drop_duplicates(subset = ['credit_card_name'])
# df.to_csv('all_card_details.csv',index=False)
df.head()

,credit_card_name,lounge_access,milestone,rewards,fees
0,MoneyBack+ Credit Card,None,None,"[10X CashPoints on Amazon, BigBasket , Flipkar...",[Joining/Renewal Membership Fee – ₹500/- + App...
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,"[5% Cashback on Amazon, BookMyShow, Cult.fit, ...","[Joining/Renewal Membership Fee – ₹1,000/- + A..."
2,IndianOil HDFC Bank Credit Card,None,None,"[Fuel Points, , , Earn up to 50 Litres of Free...",[Joining/Renewal Membership Fee – Rs. 500/- pl...
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",[Complimentary Club Vistara Silver Tier and MM...,[Joining/ Renewal Membership Fee of Rs. 2500 +...
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,"[HDFC Bank Reward Points, 5 Reward Point for e...",[Joining/Renewal Membership Fee – Rs. 500/- + ...


In [59]:
card_fees = []
card_rev_cond = []
for s in df.fees.values:
    s = set(s)
    M = []
    for i in s:
        M.extend(i.split('^^'))
    
    reversal_cond = list(filter(lambda x:True if re.search(r'waiv\w*|membership\s*\w*\s*free|spend\w*',x.lower()) else False, M))
    reversal_cond = list(map(lambda x:re.sub(r'(\xa0)*|(\u200b)*',"",x).strip(), reversal_cond))
    
    if len(reversal_cond)!=0:
        card_rev_cond.append(';'.join(reversal_cond))
    else:
        card_rev_cond.append('None')
    
    
    M = list(filter(lambda x:True if re.search(r'\d+',x.lower()) else False, M))
    
    M = list(filter(lambda x:False if re.search(r'waiv\w+',x.lower()) else True, M))
    
    pattern = r'nominal|join\w*|minimum|renewal membership|per annum|monthly membership fee|annual membership fee'
    fees = list(filter(lambda x:True if re.search(pattern,x.lower()) else False, M))
    
    if len(fees)!=0:
        card_fees.append(';'.join(fees))
    else:
        card_fees.append('None')

In [60]:

df['card_fees'] = card_fees
df['card_rev_cond'] = card_rev_cond
df.head()

,credit_card_name,lounge_access,milestone,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,"[10X CashPoints on Amazon, BigBasket , Flipkar...",[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,"[5% Cashback on Amazon, BookMyShow, Cult.fit, ...","[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,"[Fuel Points, , , Earn up to 50 Litres of Free...",[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",[Complimentary Club Vistara Silver Tier and MM...,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,"[HDFC Bank Reward Points, 5 Reward Point for e...",[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


In [61]:
df.rewards.values

array([list(['10X CashPoints on Amazon, BigBasket , Flipkart, Reliance Smart SuperStore & Swiggy', '5X CashPoints on EMI spends at merchant locations', '2 CashPoints per ₹150 spent on other spends ( Excluding fuel, wallets loads/prepaid card loads & voucher purchases )', 'Get\xa0₹500 gift voucher on spends on\xa0₹50,000 per calendar quarter', 'Upto 20% discount on partner restaurants via Swiggy Dineout', 'Note:', 'With effect from 1st January 2023, Rent payments and Govt Related transactions will NOT earn Reward Points', 'With effect from 1st January 2023, Reward points earned on Grocery transactions will be capped per month to 1,000 reward points.', 'For detailed Terms and conditionsClick here', 'Zero lost Card liability :In the unfortunate event of losing your HDFC Bank Moneyback+ Credit Card, report it immediately to our 24-hour call centre. On reporting the loss immediately, you have zero liability on any fraudulent transactions made on your Credit Card.', 'Interest Free Credit Per

In [62]:
rew_val = []
for rew in  df.rewards.values:
    if rew != None:
        
        M = list(filter(lambda x:True if re.search(r'\d+x|\d+%|\d+\s+',x.lower()) else False, rew))
        M = set(M)
        
        M = list(filter(lambda x:False if re.search(r'(rewardpoint|rp) =',x.lower()) else True, M))
        M = list(filter(lambda x:True if re.search(r'rewards|point|reward point\w*|cashpoints|spends',x.lower()) else False, M))
        
        M = set(map(lambda x:x.replace('\xa0','').replace('\u200b',' ').strip(), M))
        M = list(l)
        
        M = list(filter(lambda x:False if re.search(r'redeem|redemption|more|capped',x.lower()) else True, M))
        M = list(filter(lambda x:False if re.search(r'\w*with effect\w*',x.lower()) else True, M))
        
        
        if len(M)!=0:
            rew_val.append(';'.join(M))
        else:
            rew_val.append('None')
    else:
        rew_val.append('None')


df.rewards = rew_val
df.head()

,credit_card_name,lounge_access,milestone,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,None,[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,None,"[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,None,[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",None,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,None,[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


In [63]:
As = []
for i in df.lounge_access.values:
    
    M = list(filter(lambda x:True if re.search(r'\d+|unlimited',x.lower()) else False, i))
    
    
    M = list(filter(lambda x:False if re.search(r'\s*\w*click\w*\s*',x.lower()) else True, M))
    
    M = list(filter(lambda x:True if re.search(r'lounge access|complimentary',x.lower()) else False, M))
    
    M = set(map(lambda x:x.replace('\xa0','').replace('\u200b',' ').strip(), M))
    M = list(M)
    if len(M)!=0:
        As.append(';'.join(M))
    else:
        As.append('None')

df.lounge_access = As


In [64]:
values = []
for mb in df.milestone.values:
    if mb!='None':
        M = set(map(lambda x:x.replace('\xa0','').replace('\u200b',' ').strip(), mb))
        M = list(filter(lambda x:False if re.search(r'\s*\w*click\w*\s*',x.lower()) else True, M))
        values.append(';'.join(M))
    else:
        values.append('None')
df.milestone = values

In [65]:
df.head()

,credit_card_name,lounge_access,milestone,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,None,[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,None,None,None,"[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,None,[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,"Using Priority Pass, you and your add-on membe...",";Get additional ₹ 5,000 worth flight vouchers ...",None,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,None,,None,[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


In [66]:

df = df.drop_duplicates(subset=['credit_card_name'])
df = df[['credit_card_name','card_fees','rewards','lounge_access', 'milestone', 'card_rev_cond']]
df.head()

,credit_card_name,card_fees,rewards,lounge_access,milestone,card_rev_cond
0,MoneyBack+ Credit Card,Joining/Renewal Membership Fee – ₹500/- + Appl...,None,None,None,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,"Joining/Renewal Membership Fee – ₹1,000/- + Ap...",None,None,None,"Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,Joining/Renewal Membership Fee – Rs. 500/- plu...,None,None,None,None
3,Regalia Gold Credit Card,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None,"Using Priority Pass, you and your add-on membe...",";Get additional ₹ 5,000 worth flight vouchers ...",None
4,IRCTC HDFC Bank Credit Card,Joining/Renewal Membership Fee – Rs. 500/- + A...,None,None,,"Spend Rs. 1,50,000 or more in a year, before y..."


In [67]:
df.to_csv('bayesian_technology_credit_card_data_scrap.csv',index=False)